# 〇. 写在前面
## 0.1 参考来源
1. https://blog.csdn.net/weixin_46098577/article/details/120167360

In [29]:
import open3d as o3d
import numpy as np
import pythreejs as p3
from IPython.display import display

In [28]:
import os
import sys
# 获取当前Notebook所在目录
current_dir = os.path.dirname(os.path.abspath('__file__'))

# 构造自建库路径
lib_path = os.path.join(current_dir, '../../utils_lee')
# 将自建库路径添加到系统路径中
sys.path.append(lib_path)

# from image_utils import ImageUtils
# from file_utils import FileUtils
from point_cloud_utils import PointCloudUtils

In [8]:
PointCloudUtils.render_point_cloud_with_axes(file_path="./bumpy_plane_test_4.ply", point_size=0.0001, canvas_dimensions=(900, 400), show_light=True)

Renderer(camera=PerspectiveCamera(aspect=2.25, fov=75.0, position=(0.0, 0.0, 3.0), projectionMatrix=(1.0, 0.0,…

# 〇. 点云文件地址

In [21]:
ply1_path = "./artifact19.ply"
ply1_pcd = o3d.io.read_point_cloud(ply1_path)

ply2_path = "./bumpy_plane_test_4.ply"
ply2_pcd = o3d.io.read_point_cloud(ply2_path)

off_path = "./bookshelf_0574.off"

# 一. 点云可视化
## `o3d.visualization.draw_geometries` 

### 基本用法

```python
o3d.visualization.draw_geometries([geometry0, geometry1, ...], window_name='Open3D', width=800, height=600, point_show_normal=False, mesh_show_wireframe=False, mesh_show_back_face=False)
```

### 参数说明
| 参数名称 | 类型 | 描述 | 默认值 |
| :--: | :--: | :--: | :--: |
| geometries | list of `o3d.geometry.Geometry3D` | 包含要可视化的3D几何体的列表。可以是点云、网格等。 | 无（必须提供） |
| window_name | str | 显示窗口的名称。 | 'Open3D' |
| width | int | 窗口的宽度（以像素为单位）。 | 800 |
| height | int | 窗口的高度（以像素为单位）。 | 600 |
| point_show_normal | bool | 如果为True，则在点云中显示每个点的法线。 | False |
| mesh_show_wireframe | bool | 如果为True，则在网格上绘制线框。 | False |
| mesh_show_back_face | bool | 如果为True，则渲染网格的背面。 | False |

In [3]:
import open3d as o3d

# 读取PLY文件
ply_file = ply1_path
point_cloud = o3d.io.read_point_cloud(ply_file)

# 打印点云数据
print(point_cloud)
# 可视化点云
o3d.visualization.draw_geometries([point_cloud])

# 保存可见表面点云为ply文件
# o3d.io.write_point_cloud("visible_surface.ply", point_cloud)

PointCloud with 187114 points.


In [17]:
import open3d as o3d

# 创建一个点云
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector([[0, 0, 0], [1, 0, 0], [0, 1, 0]])

# 创建 Visualizer
vis = o3d.visualization.Visualizer()
vis.create_window()

# 添加点云到 Visualizer
vis.add_geometry(pcd)

# 获取相机参数
ctr = vis.get_view_control()
param = ctr.convert_to_pinhole_camera_parameters()

# 设置相机内参（例如，焦距、主点等）
param.intrinsic.set_intrinsics(width=800, height=600, fx=400, fy=400, cx=400, cy=300)

# 设置相机外参（例如，相机的位置和朝向）
param.extrinsic = o3d.core.Tensor([[1, 0, 0, 0], 
                                   [0, 1, 0, 0], 
                                   [0, 0, 1, -3], 
                                   [0, 0, 0, 1]])

# 应用新的相机参数
ctr.convert_from_pinhole_camera_parameters(param)

# 运行可视化
vis.run()

# 关闭窗口
vis.destroy_window()


TypeError: (): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.camera.PinholeCameraParameters, arg0: numpy.ndarray[numpy.float64[4, 4]]) -> None

Invoked with: PinholeCameraParameters class.
Access its data via intrinsic and extrinsic., [[1 0 0 0],
 [0 1 0 0],
 [0 0 1 -3],
 [0 0 0 1]]
Tensor[shape={4, 4}, stride={4, 1}, Int64, CPU:0, 0x12a6f1710]

# 二. 下采样

In [9]:
import open3d as o3d
import numpy as np

print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud(ply1_path)
print(pcd)

print("->正在可视化原始点云")
o3d.visualization.draw_geometries([pcd])

print("->正在体素下采样...")
voxel_size = 0.5
downpcd = pcd.voxel_down_sample(voxel_size)
print(downpcd)

print("->正在可视化下采样点云")
o3d.visualization.draw_geometries([downpcd])


->正在加载点云... 
PointCloud with 187114 points.
->正在体素下采样...
PointCloud with 109 points.
->正在可视化下采样点云


# x.空间变换

## x.1 平移

### x.1.1 上下左右平移

In [12]:
import copy  # 点云深拷贝
import open3d as o3d

# -------------------------- 加载点云 ------------------------
print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud(ply1_path)
print(pcd)
print(f'pcd质心: {pcd.get_center()}')
# ===========================================================

# -------------------------- 点云平移 ------------------------
print("\n->沿X轴平移0.2m")
pcd_tx = copy.deepcopy(pcd).translate((0.2, 0, 0))
pcd_tx.paint_uniform_color([1, 0, 0])
print(pcd_tx)
print(f'pcd_tx质心: {pcd_tx.get_center()}')

print("\n->沿Y轴平移0.2m")
pcd_ty = copy.deepcopy(pcd_tx).translate((0, 0.2, 0))
pcd_ty.paint_uniform_color([0, 1, 0])
print(pcd_ty)
print(f'pcd_ty质心: {pcd_ty.get_center()}')

print("\n->沿X轴平移-0.2m, 再沿Y轴平移0.2m")
pcd_txy = copy.deepcopy(pcd).translate((-1, 0.2, 0))
pcd_txy.paint_uniform_color([0, 0, 1])
print(pcd_txy)
print('pcd_txy质心: ', pcd_txy.get_center())
# ===========================================================

# -------------------------- 可视化 --------------------------
o3d.visualization.draw_geometries([pcd, pcd_tx, pcd_ty, pcd_txy])
# ===========================================================


->正在加载点云... 
PointCloud with 187114 points.
pcd质心: [-0.11644541  0.04417858 -1.29565716]

->沿X轴平移0.2m
PointCloud with 187114 points.
pcd_tx质心: [ 0.08355459  0.04417858 -1.29565716]

->沿Y轴平移0.2m
PointCloud with 187114 points.
pcd_ty质心: [ 0.08355459  0.24417858 -1.29565716]

->沿X轴平移-0.2m, 再沿Y轴平移0.2m
PointCloud with 187114 points.
pcd_txy质心:  [-1.11644541  0.24417858 -1.29565716]


### x.1.2 将点云质心平移到指定位置

In [14]:
import copy  # 点云深拷贝
import open3d as o3d

# -------------------------- 加载点云 ------------------------
print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud(ply1_path)
print(pcd)
print(f'pcd质心: {pcd.get_center()}')
# ===========================================================

# -------------------------- 点云平移 ------------------------
print("\n->沿X轴平移0.2m")
pcd_tx = copy.deepcopy(pcd).translate((0.2, 0.2, 0.2))
pcd_tx.paint_uniform_color([1, 0, 0])
print(pcd_tx)
print(f'pcd_tx质心: {pcd_tx.get_center()}')

print("\n->将点云质心平移到指定位置")
pcd_new = copy.deepcopy(pcd_tx).translate((10, 10, 10),relative=False)
pcd_new.paint_uniform_color([0, 1, 0])
print(pcd_new)
print(f'pcd_new: {pcd_new.get_center()}')
# ===========================================================

# -------------------------- 可视化 --------------------------
o3d.visualization.draw_geometries([pcd, pcd_tx, pcd_new])
# ===========================================================


->正在加载点云... 
PointCloud with 187114 points.
pcd质心: [-0.11644541  0.04417858 -1.29565716]

->沿X轴平移0.2m
PointCloud with 187114 points.
pcd_tx质心: [ 0.08355459  0.24417858 -1.09565716]

->将点云质心平移到指定位置
PointCloud with 187114 points.
pcd_new: [10. 10. 10.]


## x.2 旋转

In [22]:
import open3d as o3d
import numpy as np
import copy

# 生成旋转矩阵 (这里以绕 Z 轴旋转 90 度为例)
R = ply2_pcd.get_rotation_matrix_from_xyz((0, 0, np.pi / 2))

# 复制并旋转点云
pcd_new = copy.deepcopy(ply2_pcd).rotate(R, center=(0, 0, 0))

# 可视化原点云和旋转后的点云
o3d.visualization.draw_geometries([ply2_pcd, pcd_new], window_name="点云旋转", width=800, height=600)


### x.2.1 用欧垃角

In [40]:
# -*- coding: utf-8 -*-
"""
Created on Sat Apr 16 08:51:41 2022
@author: https://blog.csdn.net/suiyingy
"""
# from mayavi import mlab
import numpy as np
import open3d as o3d
 
def ply_read(file_path):
    lines = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
    return lines
 
 
#将每一行数据分割后转为数字
def ls2n(line):
    line = line.strip().split(' ')
    return list(map(float, line))
 
 
def viz_mayavi_3(points1, points2, points3):
    PointCloudUtils.render_point_cloud_with_axes(points_array=points1, point_size=0.0001, canvas_dimensions=(900, 400), show_light=True)
    PointCloudUtils.render_point_cloud_with_axes(points_array=points2, point_size=0.0001, canvas_dimensions=(900, 400), show_light=True)
    PointCloudUtils.render_point_cloud_with_axes(points_array=points3, point_size=0.0001, canvas_dimensions=(900, 400), show_light=True)
       
if __name__ == '__main__':
    file_path = 'bumpy_plane_test_4.ply'
    points1 = PointCloudUtils.load_point_cloud(file_path='./bumpy_plane_test_4.ply')["points"]
   
    file_path = 'bumpy_plane_test_4.ply'
    points2 = PointCloudUtils.load_point_cloud(file_path='./bumpy_plane_test_4.ply')["points"]
   
    threshold = 0.5 #距离阈值
    trans_init = np.array([[1.0, 0.0, 0.0, 0.0],
                           [0.0, 1.0, 0.0, 0.0],
                           [0.0, 0.0, 1.0, 0],
                           [0.0, 0.0, 0.0,   1.0]])
    #计算两个重要指标，fitness计算重叠区域（内点对应关系/目标点数）。越高越好。
    #inlier_rmse计算所有内在对应关系的均方根误差RMSE。越低越好。
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(points1)
    target = o3d.geometry.PointCloud()
    target.points = o3d.utility.Vector3dVector(points2)

   # 初始对齐
    print("Initial alignment")
    print(source)
    icp = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    print(icp)

    # 应用变换
    source.transform(icp.transformation)
    points3 = np.array(source.points)

    # 可视化结果
    viz_mayavi_3(points1, points2, points3)



Initial alignment
PointCloud with 10000 points.
RegistrationResult with fitness=1.000000e+00, inlier_rmse=9.879593e-16, and correspondence_set size of 10000
Access transformation to get result.


Renderer(camera=PerspectiveCamera(aspect=2.25, fov=75.0, position=(0.0, 0.0, 3.0), projectionMatrix=(1.0, 0.0,…

Renderer(camera=PerspectiveCamera(aspect=2.25, fov=75.0, position=(0.0, 0.0, 3.0), projectionMatrix=(1.0, 0.0,…

Renderer(camera=PerspectiveCamera(aspect=2.25, fov=75.0, position=(0.0, 0.0, 3.0), projectionMatrix=(1.0, 0.0,…

: 